In [0]:
-- GOLD: curated user dimension with derived features and no-null outputs
CREATE OR REFRESH STREAMING TABLE users_gold
COMMENT "Curated user dimension for analytics/ML; normalized, enriched, and no-null outputs"
AS
WITH base AS (
  SELECT
    /* Keys & stable identifiers */
    CAST(id AS STRING)                           AS user_id,

    /* Original clean fields from silver */
    TRIM(name)                                   AS name,
    LOWER(TRIM(username))                        AS username,
    LOWER(TRIM(email))                           AS email,
    TRIM(phone)                                  AS phone,
    LOWER(TRIM(gender))                          AS gender,
    LOWER(TRIM(food_preference))                 AS food_preference,
    LOWER(TRIM(locality))                        AS locality,          -- normalized for grouping
    TRIM(address)                                AS address,
    LOWER(TRIM(role))                            AS role,
    CAST(is_active AS BOOLEAN)                   AS is_active,
    CAST(created_at AS TIMESTAMP)                AS created_at,
    CAST(updated_at AS TIMESTAMP)                AS updated_at,

    /* -------- Derived attributes (analytics-friendly) -------- */

    /* Email domain (for channel performance, deliverability insights) */
    CASE
      WHEN email LIKE '%@%' THEN LOWER(SPLIT(email, '@')[1])
      ELSE 'unknown'
    END                                          AS email_domain,

    /* Locality slug (remove spaces/specials for dimensional grouping) */
    REGEXP_REPLACE(LOWER(TRIM(locality)), '[^a-z0-9]+', '_') AS locality_slug,

    /* Role flags (for quick slicing in BI/ML) */
    CASE WHEN LOWER(role) = 'admin'    THEN TRUE ELSE FALSE END       AS is_admin,
    CASE WHEN LOWER(role) = 'partner'  THEN TRUE ELSE FALSE END       AS is_partner,
    CASE WHEN LOWER(role) = 'customer' THEN TRUE ELSE FALSE END       AS is_customer,

    /* Activity windows (7/30/90 days) relative to updated_at */
    DATE_SUB(CAST(updated_at AS DATE), 7)   AS activity_window_7d_start,
    DATE_SUB(CAST(updated_at AS DATE), 30)  AS activity_window_30d_start,
    DATE_SUB(CAST(updated_at AS DATE), 90)  AS activity_window_90d_start,

    /* Recency metrics */
    DATEDIFF(CAST(updated_at AS DATE), CAST(created_at AS DATE))      AS days_since_signup,
    DATEDIFF(CURRENT_DATE(), CAST(updated_at AS DATE))                AS days_since_last_update,

    /* Simple engagement segment (edit the thresholds to your needs) */
    CASE
      WHEN is_active = TRUE AND DATEDIFF(CURRENT_DATE(), CAST(updated_at AS DATE)) <= 7  THEN 'active_7d'
      WHEN is_active = TRUE AND DATEDIFF(CURRENT_DATE(), CAST(updated_at AS DATE)) <= 30 THEN 'active_30d'
      WHEN is_active = TRUE AND DATEDIFF(CURRENT_DATE(), CAST(updated_at AS DATE)) <= 90 THEN 'active_90d'
      WHEN is_active = TRUE THEN 'active_gt_90d'
      ELSE 'inactive'
    END                                          AS engagement_segment
  FROM STREAM(users_silver)
),

-- Final projection with COALESCE defaults to ensure no-null outputs for BI
final AS (
  SELECT
    /* Keys */
    user_id,

    /* Core identity */
    COALESCE(name,           'unknown')          AS name,
    COALESCE(username,       'unknown')          AS username,
    COALESCE(email,          'unknown@unknown')  AS email,
    COALESCE(email_domain,   'unknown')          AS email_domain,

    /* Contact / profile */
    COALESCE(phone,          'unknown')          AS phone,
    COALESCE(gender,         'unknown')          AS gender,
    COALESCE(food_preference,'unknown')          AS food_preference,

    /* Location */
    COALESCE(locality,       'unknown')          AS locality,
    COALESCE(locality_slug,  'unknown')          AS locality_slug,
    COALESCE(address,        'unknown')          AS address,

    /* Access/role */
    COALESCE(role,           'unknown')          AS role,
    COALESCE(is_admin,       FALSE)              AS is_admin,
    COALESCE(is_partner,     FALSE)              AS is_partner,
    COALESCE(is_customer,    FALSE)              AS is_customer,

    /* Lifecycle & activity */
    COALESCE(is_active,      FALSE)              AS is_active,
    COALESCE(created_at,     TIMESTAMP('1970-01-01 00:00:00')) AS created_at,
    COALESCE(updated_at,     CURRENT_TIMESTAMP())              AS updated_at,

    /* Windows & recency */
    COALESCE(activity_window_7d_start,  DATE('1970-01-01')) AS activity_window_7d_start,
    COALESCE(activity_window_30d_start, DATE('1970-01-01')) AS activity_window_30d_start,
    COALESCE(activity_window_90d_start, DATE('1970-01-01')) AS activity_window_90d_start,
    COALESCE(days_since_signup,         0)                  AS days_since_signup,
    COALESCE(days_since_last_update,    9999)               AS days_since_last_update,

    /* Engagement */
    COALESCE(engagement_segment,        'unknown')          AS engagement_segment
  FROM base
)

SELECT * FROM final;


In [0]:
CREATE OR REFRESH STREAMING TABLE orders_gold
COMMENT "Curated order dimension for analytics/ML; normalized, enriched, and no-null outputs"
AS
WITH base AS (
  SELECT
    /* Keys & stable identifiers */
    CAST(order_id AS STRING)                      AS order_id,
    CAST(user_id AS STRING)                       AS user_id,

    /* Original clean fields from silver */
    COALESCE(total_amount, 0)                     AS total_amount,
    COALESCE(items_count, 0)                      AS items_count,
    TRIM(restaurant_name)                         AS restaurant_name,
    CAST(timestamp AS TIMESTAMP)                  AS order_timestamp,

    /* -------- Derived attributes -------- */

    /* Revenue bucket for quick BI segmentation */
    CASE
      WHEN total_amount < 100 THEN 'low_value'
      WHEN total_amount BETWEEN 100 AND 500 THEN 'medium_value'
      ELSE 'high_value'
    END                                           AS revenue_segment,

    /* Order recency metrics */
    DATEDIFF(CURRENT_DATE(), CAST(timestamp AS DATE)) AS days_since_order,

    /* Restaurant slug for grouping */
    REGEXP_REPLACE(LOWER(TRIM(restaurant_name)), '[^a-z0-9]+', '_') AS restaurant_slug,

    /* Duplicate flag for ML */
    CASE WHEN has_duplicate = 'true' THEN 1 ELSE 0 END AS duplicate_flag
  FROM STREAM(orders_silver)
),

final AS (
  SELECT
    /* Keys */
    COALESCE(order_id, 'unknown')                 AS order_id,
    COALESCE(user_id, 'unknown')                  AS user_id,

    /* Core order details */
    COALESCE(restaurant_name, 'unknown')          AS restaurant_name,
    COALESCE(restaurant_slug, 'unknown')          AS restaurant_slug,

    /* Metrics */
    COALESCE(total_amount, 0)                     AS total_amount,
    COALESCE(items_count, 0)                      AS items_count,

    /* Derived segments */
    COALESCE(revenue_segment, 'unknown')          AS revenue_segment,

    /* Flags */
    COALESCE(duplicate_flag, 0)                   AS duplicate_flag,

    /* Timestamps & recency */
    COALESCE(order_timestamp, TIMESTAMP('1970-01-01 00:00:00')) AS order_timestamp,
    COALESCE(days_since_order, 9999)              AS days_since_order
  FROM base
)

SELECT * FROM final;